In [1]:
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 98378, done.
remote: Counting objects: 100% (884/884), done.
remote: Compressing objects: 100% (476/476), done.
remote: Total 98378 (delta 477), reused 730 (delta 385), pack-reused 97494 (from 1)
Receiving objects: 100% (98378/98378), 621.96 MiB | 19.74 MiB/s, done.
Resolving deltas: 100% (71461/71461), done.


In [2]:
from __future__ import print_function
from IPython import display
base_name = checkpoint_name = 'mobilenet_v2_1.0_224' #@param
url = 'https://storage.googleapis.com/mobilenet_v2/checkpoints/' + checkpoint_name + '.tgz'
print('Downloading from ', url)
!wget {url}
print('Unpacking')
!tar -xvf {checkpoint_name}.tgz
checkpoint = checkpoint_name + '.ckpt'

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint)


Successfully downloaded checkpoint from  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz . It is available as mobilenet_v2_1.0_224.ckpt


In [3]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

--2024-11-07 03:34:50--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.1s    

2024-11-07 03:34:50 (928 KB/s) - ‘panda.jpg’ saved [116068/116068]



In [4]:
import sys
sys.path.append('/content/models/research/slim')

# Install the tf_slim package
!pip install tf_slim

Checkpoint based inference

In [5]:
import tensorflow as tf
import numpy as np

# Load the MobileNetV2 model from tf.keras (no top layers, for feature extraction)
mobilenet_v2_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Set the model to inference mode (no training)
mobilenet_v2_model.trainable = False

# Example of a random image (replace with actual image loading)
images = np.random.random((1, 224, 224, 3))

# Get feature maps from the model (equivalent to 'endpoints' in tf-slim)
features = mobilenet_v2_model(images)

print(features.shape)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(1, 7, 7, 1280)


In [6]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions

# Load the MobileNetV2 model pre-trained on ImageNet
mobilenet_v2_model = MobileNetV2(weights='imagenet', include_top=True)

# Load and preprocess an image for prediction
img_path = 'panda.jpg'  # replace with your image path
img = Image.open(img_path)
img = img.resize((224, 224))  # resize to 224x224 as expected by MobileNetV2
img_array = np.array(img) / 255.0  # normalize the image


# Check if eager execution is enabled
print("Eager execution:", tf.executing_eagerly())

# Convert the image to a TensorFlow tensor (in case it's necessary)
img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)


# Ensure eager execution is enabled before any other TensorFlow operations
tf.compat.v1.enable_eager_execution()

# Check if eager execution is enabled
print("Eager execution:", tf.executing_eagerly())

# Make a prediction using the model
predictions = mobilenet_v2_model.predict(img_array)

# Decode predictions to human-readable labels
decoded_predictions = decode_predictions(predictions, top=3)[0]

# Print top 3 predictions
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i+1}: {label} ({score*100:.2f}%)")


In [ ]:
import numpy as np
img = np.array(PIL.Image.open('panda.jpg').resize((224, 224))).astype(np.float) / 128 - 1
gd = tf.GraphDef.FromString(open(checkpoint_name + '_frozen.pb', 'rb').read())
inp, predictions = tf.import_graph_def(gd,  return_elements = ['input:0', 'MobilenetV2/Predictions/Reshape_1:0'])

In [ ]:
with tf.Session(graph=inp.graph):
  x = predictions.eval(feed_dict={inp: img.reshape(1, 224,224, 3)})

label_map = imagenet.create_readable_names_for_imagenet_labels()
print("Top 1 Prediction: ", x.argmax(),label_map[x.argmax()], x.max())